<a href="https://colab.research.google.com/github/ReaganJHarris/ET_Retrieval/blob/master/ESPA_Order_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ESPA-API DEMO code

Since many of our services written in python also interact with the API, we have
this example as a quick run-through which should hopefully get anyone started
towards building their own simple python services capable of interacting
with ESPA.

## Official documentation:
* See the [ESPA API Source Code](https://github.com/USGS-EROS/espa-api/)
* Visit the [ESPA On-Demand Interface](https://espa.cr.usgs.gov)

For questions regarding this source code, or the ESPA project, please use the
[Landsat Contact Us](https://landsat.usgs.gov/contact) page and specify
**USGS ESPA** in the "Subject" section.

### WARNING! _This example is only provided as is._

---

---

In [ ]:
import platform
print(platform.python_version())

3.7.10


## Dependencies
We will use the [requests](http://docs.python-requests.org/en/master/)
library, although similar operations are available through the
[Standard Python Libraries](https://docs.python.org/2/library/internet.html)

In [ ]:
import os
import requests
import json
import getpass

import urllib.request

In [ ]:
from  xarray import open_rasterio


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
!pip install -U rasterio

     |████████████████████████████████| 19.3MB 1.5MB/s 


In [ ]:
# !pip install --force-reinstall Affine

In [ ]:
# import affine

The current URL hosting the ESPA interfaces has reached a stable version 1.0

In [ ]:
host = 'https://espa.cr.usgs.gov/api/v1/'

ESPA uses the ERS credentials for identifying users

In [ ]:
username = 'earth_explorer_username'
password = getpass.getpass()

··········


---

---

## espa_api: A Function
First and foremost, define a simple function for interacting with the API. 

The key things to watch for:

* Always scrub for a `"messages"` field returned in the response, it is only informational about a request
  * **Errors** (`"errors"`): Brief exlaination about why a request failed
  * **Warnings** (`"warnings"`): Cautions about a successful response
* Always make sure the requested HTTP `status_code` returned is valid 
  * **GET**: `200 OK`: The requested resource was successfully fetched (result can still be empty)
  * **POST**: `201 Created`: The requested resource was created
  * **PUT**: `202 Accepted`: The requested resource was updated

In [ ]:
def espa_api(endpoint, verb='get', body=None, uauth=None):
    """ Suggested simple way to interact with the ESPA JSON REST API """
    auth_tup = uauth if uauth else (username, password)
    response = getattr(requests, verb)(host + endpoint, auth=auth_tup, json=body)
    print('{} {}'.format(response.status_code, response.reason))
    data = response.json()
    if isinstance(data, dict):
        messages = data.pop("messages", None)  
        if messages:
            print(json.dumps(messages, indent=4))
    try:
        response.raise_for_status()
    except Exception as e:
        print(e)
        return None
    else:
        return data

## General Interactions: Authentication
Basic call to get the current user's information. It requires valid credentials, and is a good check that the system is available

In [ ]:
print('GET /api/v1/user')
resp = espa_api('user')
print(json.dumps(resp, indent=4))

Here, we can see what an error response will look like:

In [ ]:
print('GET /api/v1/user')
espa_api('user', uauth=('invalid', 'invalid'))

## General Interactions: Available Options

ESPA offers several services, descriptions can be found here: 
* [AVAILABLE-PRODUCTS](/docs/available-products.md)
* [CUSTOMIZATION](/docs/customization.md)

Call to demonstrate what is returned from available-products

In [ ]:
print('GET /api/v1/available-products')
# avail_list = {'inputs': ['LC08_L1TP_034032_20180102_20180118_01_T1'
#                         ]
#              }

avail_list = {'inputs': ['LE07_L1TP_033032_20130427_20160908_01_T1'
                        ]
             }

resp = espa_api('available-products', body=avail_list)
print(json.dumps(resp, indent=4))

ESPA can produce outputs all of the same geographic projections.  

Call to show the available projection parameters that can be used:

In [ ]:
print('GET /api/v1/projections')
projs = espa_api('projections')
print(type(projs.keys()))
print(projs.keys())
#print(json.dumps(projs.keys()))
print(json.dumps(projs))

This is a Schema Definition, useful for building a valid order

Example (*UTM Projection*):

In [ ]:
print(json.dumps(projs['utm']['properties'], indent=4))

In [ ]:
#print(json.dumps(projs['utm']['properties'], indent=4))

### More resources about the API

For further reading: 

* [API-REQUIREMENTS](/docs/api-requirements.md)
* [API-RESOURCES-LIST](/docs/api-resources-list.md)
* [Product Flow](/docs/product_flow.txt)
* [TERMS](/docs/terms.md)

---

---

## Practical Example: Building An Order
Here we use two different Landsat sensors to build up an order, and then place the order into the system

In [ ]:
# https://earthexplorer.usgs.gov/
# Dataset: Landsat 8 OLI/TIRS C1 Level-2


In [ ]:
l8_ls = ['LC08_L1TP_032032_20180104_20180118_01_T1',
'LC08_L1TP_032033_20180104_20180118_01_T1',
'LC08_L1TP_032032_20180120_20180206_01_T1',
'LC08_L1TP_032033_20180120_20180206_01_T1',
'LC08_L1TP_032032_20180205_20180221_01_T1',
'LC08_L1TP_032033_20180205_20180221_01_T1',
'LC08_L1TP_032032_20180221_20180308_01_T1',
'LC08_L1TP_032033_20180221_20180308_01_T1',
'LC08_L1TP_032032_20180309_20180320_01_T1',
'LC08_L1TP_032033_20180309_20180320_01_T1',
'LC08_L1TP_032032_20180325_20180404_01_T1',
'LC08_L1TP_032033_20180325_20180404_01_T1',
'LC08_L1TP_032032_20180410_20180417_01_T1',
'LC08_L1TP_032033_20180410_20180417_01_T1',
'LC08_L1TP_032032_20180426_20180502_01_T1',
'LC08_L1TP_032033_20180426_20180502_01_T1',
'LC08_L1TP_032033_20180512_20180517_01_T1',
'LC08_L1TP_032033_20180528_20180605_01_T1',
'LC08_L1TP_032032_20180613_20180703_01_T1',
'LC08_L1TP_032033_20180613_20180703_01_T1',
'LC08_L1TP_032032_20180629_20180716_01_T1',
'LC08_L1TP_032033_20180629_20180716_01_T1',
'LC08_L1TP_032032_20180715_20180730_01_T1',
'LC08_L1TP_032033_20180715_20180730_01_T1',
'LC08_L1TP_032032_20180731_20180814_01_T1',
'LC08_L1TP_032033_20180731_20180814_01_T1',
'LC08_L1TP_032032_20180816_20180829_01_T1',
'LC08_L1TP_032033_20180816_20180829_01_T1',
'LC08_L1TP_032032_20180901_20180912_01_T1',
'LC08_L1TP_032033_20180901_20180912_01_T1',
'LC08_L1TP_032032_20180917_20180928_01_T1',
'LC08_L1TP_032033_20180917_20180928_01_T1',
'LC08_L1TP_032032_20181003_20181010_01_T1',
'LC08_L1TP_032033_20181003_20181010_01_T1',
'LC08_L1TP_032032_20181019_20181031_01_T1',
'LC08_L1TP_032033_20181019_20181031_01_T1',
'LC08_L1TP_032032_20181104_20181115_01_T1',
'LC08_L1TP_032033_20181104_20181115_01_T1',
'LC08_L1TP_032032_20181120_20181129_01_T1',
'LC08_L1TP_032033_20181120_20181129_01_T1',
'LC08_L1TP_032033_20181222_20181227_01_T1'

        ]
         

# l5_ls = ['LT05_L1TP_033032_20110516_20160901_01_T1']


# Differing products across the sensors
# l7_prods = ['et']
# l5_prods = ['et']
#l8_prods = ['s', 'r']
l8_prods = ['et']

# Standard Albers CONUS
# projection = {'aea': {'standard_parallel_1': 29.5,
#                       'standard_parallel_2': 45.5,
#                       'central_meridian': -96.0,
#                       'latitude_of_origin': 23.0,
#                       'false_easting': 0,
#                       'false_northing': 0,
#                       'datum': 'nad83'}}

#'datum': 'wgs84'
# UTM Zone	13
# Scene Center Latitude	40.33272
# Scene Center Longitude	-104.21059
# Corner Upper Left Latitude	41.38717
# Corner Upper Left Longitude	-105.58009
# Corner Upper Right Latitude	41.36793
# Corner Upper Right Longitude	-102.81789
# Corner Lower Left Latitude	39.27099
# Corner Lower Left Longitude	-105.56225
# Corner Lower Right Latitude	39.25314
# Corner Lower Right Longitude	-102.88495

# Let available-products place the acquisitions under their respective sensors
#ls = l8_ls + l7_ls
# ls = l7_ls + l5_ls
ls = l8_ls

print('GET /api/v1/available-products')
order = espa_api('available-products', body=dict(inputs=ls))
print(json.dumps(order, indent=4))

**NOTE**: Here we will not need to know what the sensor names were for the Product IDs, thanks to the response from this `available-products` resource. 

In [ ]:
# Replace the available products that was returned with what we want
for sensor in order.keys():
    if isinstance(order[sensor], dict) and order[sensor].get('inputs'):
        # if set(l7_ls) & set(order[sensor]['inputs']):
        #    order[sensor]['products'] = l7_prods
        if set(l8_ls) & set(order[sensor]['inputs']):
            order[sensor]['products'] = l8_prods
        # if set(l5_ls) & set(order[sensor]['inputs']):
        #     order[sensor]['products'] = l5_prods

# Add in the rest of the order information
# order['projection'] = projection
order['format'] = 'gtiff'
order['resampling_method'] = 'cc'
order['note'] = 'API Demo Jupyter!!'

# Notice how it has changed from the original call available-products
print(json.dumps(order, indent=4))

#### Place the order

In [ ]:
# Place the order
print('POST /api/v1/order')
resp = espa_api('order', verb='post', body=order)
print(json.dumps(resp, indent=4))

If successful, we will get our order-id

In [ ]:
orderid = resp['orderid']
print(orderid)

In [ ]:
orderid = ''

## Check the status of an order


In [ ]:
print('GET /api/v1/order-status/{}'.format(orderid))
resp = espa_api('order-status/{}'.format(orderid))
print(json.dumps(resp, indent=4))

Now, we can check for any completed products, and get the download url's for completed scenes

In [ ]:
print('GET /api/v1/item-status/{0}'.format(orderid))
resp = espa_api('item-status/{0}'.format(orderid), body={'status': 'complete'})
print(json.dumps(resp[orderid], indent=4))

In [ ]:
# Once the order is completed or partially completed, can get the download url's
for item in resp[orderid]:
    print("URL: {0}".format(item.get('product_dload_url')))

# Find previous orders 

List backlog orders for the authenticated user.

In [ ]:
print('GET /api/v1/list-orders')
filters = {"status": ["complete", "ordered"]}  # Here, we ignore any purged orders
resp = espa_api('list-orders', body=filters)
print(json.dumps(resp, indent=4))

## Emergency halt an Order
### PLEASE BE CAREFUL!

ESPA processes your orders in the sequence in which they are recieved.  
You may want to remove blocking orders in your queue, to prioritize your latest orders

In [ ]:
# In-process orders
print('GET /api/v1/list-orders')
filters = {"status": ["ordered"]}
orders = espa_api('list-orders', body=filters)

# Here we cancel an incomplete order
orderid = orders[0]
cancel_request = {"orderid": orderid, "status": "cancelled"}
print('PUT /api/v1/order')
order_status = espa_api('order', verb='put', body=cancel_request)

print(json.dumps(order_status, indent=4))

In [ ]:
# orders = ["espa-reaganjh@rams.colostate.edu-06222021-150600-376",
#   "espa-reaganjh@rams.colostate.edu-06222021-145427-509",
#   "espa-reaganjh@rams.colostate.edu-06222021-144014-737",
#   "espa-reaganjh@rams.colostate.edu-06222021-143222-718",
#   "espa-reaganjh@rams.colostate.edu-06222021-142042-040"
         
# ]
for order_id in orders:
  print('GET /api/v1/list-orders')
  filters = {"status": ["ordered"]}
  orders = espa_api('list-orders', body=filters)

  # Here we cancel an incomplete order
  orderid = orders[0]
  cancel_request = {"orderid": order_id, "status": "cancelled"}
  print('PUT /api/v1/order')
  order_status = espa_api('order', verb='put', body=cancel_request)

  print(json.dumps(order_status, indent=4))

# Python Script

This notebook is available as a script for [download here](/examples/api_demo.py).

# Download data

## Google Drive


In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
# update for folder to save data
Output_FP = '/gdrive/MyDrive/ARD_ET/2018_Phillips/8'

In [ ]:
# Once the order is completed or partially completed, can get the download url's
for item in resp[orderid]:
    print("URL: {0}".format(item.get('product_dload_url')))
    FSN = os.path.basename(item.get('product_dload_url'))
    print("FSN: {}".format(FSN))
    oFWN = os.path.join(Output_FP, FSN)
    print("oFWN: {}".format(oFWN))
    print("Downloading...")
    urllib.request.urlretrieve(item.get('product_dload_url'), 
                               oFWN)

In [ ]:
!tar -xzvf /gdrive/MyDrive/ET_retrieval/validation_data/LC080330322018080701T1-SC20210218040336.tar.gz -C /gdrive/MyDrive/ET_retrieval/validation_data/

In [ ]:
da = open_rasterio('/gdrive/MyDrive/ET_retrieval/validation_data/LC08_L1TP_033032_20180807_20180815_01_T1_eta.tif')
#transform = affine.from_gdal(*da.attrs['transform']) # this is important to retain the geographic attributes from the file

In [ ]:
fig = plt.figure(figsize=(16,16))
ax = fig.add_subplot(111)
ax.imshow(da.variable.data[0])
plt.show()